Cell 1: Import thư viện

In [1]:
import math
from collections import defaultdict
from underthesea import word_tokenize  # Tokenizer tiếng Việt chuẩn


Cell 2: Dữ liệu mẫu spam/ham

In [2]:
data = [
    ("Bạn đã trúng thưởng 100 triệu! Nhấp vào link nhận ngay!", "spam"),
    ("Giảm giá 50% cho đơn hàng hôm nay", "spam"),
    ("Nhận quà tặng miễn phí, click để nhận", "spam"),
    ("Mời bạn tham dự hội thảo về AI tuần tới", "ham"),
    ("Cuộc họp nhóm lúc 14h chiều nay", "ham"),
    ("Báo cáo tài chính đã được gửi qua email", "ham")
]

Cell 3: Tiền xử lí text

In [3]:
def preprocess(text):
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip()]
    return ". ".join(lines) + "." if lines else ""

Cell 4: Chuẩn hóa dữ liệu (Tokenize)

In [4]:
def tokenize(text):
    return word_tokenize(preprocess(text.lower()), format="text").split()


# Test tokenize
sample_text = "Nhận quà tặng miễn phí, click để nhận"
print("Tokenize ví dụ:", tokenize(sample_text))

Tokenize ví dụ: ['nhận', 'quà', 'tặng', 'miễn_phí', ',', 'click', 'để', 'nhận', '.']


Cell 5: Bộ đếm Naive Bayes

In [5]:
class_counts = defaultdict(int)
word_counts = defaultdict(lambda: defaultdict(int))
total_words = defaultdict(int)

for text, label in data:
    class_counts[label] += 1
    words = tokenize(text)
    for word in words:
        word_counts[label][word] += 1
        total_words[label] += 1

print("Số email theo lớp:", dict(class_counts))
print("Tổng số từ mỗi lớp:", dict(total_words))
print("Một phần word_counts cho spam:", dict(list(word_counts['spam'].items())[:5]))

Số email theo lớp: {'spam': 3, 'ham': 3}
Tổng số từ mỗi lớp: {'spam': 31, 'ham': 26}
Một phần word_counts cho spam: {'bạn': 1, 'đã': 1, 'trúng': 1, 'thưởng': 1, '100': 1}


Cell 6: Tính xác suất

In [6]:
def predict_proba(email, verbose=True):
    words = tokenize(email)
    scores = {}
    vocab = set()
    for label in class_counts:
        for word in word_counts[label]:
            vocab.add(word)
    V = len(vocab)

    for label in class_counts:
        scores[label] = math.log(class_counts[label] / sum(class_counts.values()))
        for word in words:
            count_w = word_counts[label].get(word, 0)
            scores[label] += math.log((count_w + 1) / (total_words[label] + V))

    # Chuyển log-probability -> xác suất chuẩn hóa 0-1
    max_log = max(scores.values())
    exp_scores = {label: math.exp(scores[label] - max_log) for label in scores}
    sum_exp = sum(exp_scores.values())
    probs = {label: exp_scores[label]/sum_exp for label in exp_scores}

    if verbose:
        for label, p in probs.items():
            print(f"Xác suất {label}: {p:.4f}")

    # Lớp dự đoán
    pred_label = max(probs, key=probs.get)
    return pred_label, probs

Cell 7: Test nhiều email mẫu

In [7]:
test_emails = [
    "Nhận quà miễn phí hôm nay",
    "Cuộc họp diễn ra lúc 9h sáng",
    "Giảm giá 70% cho sản phẩm mới"
]

for e in test_emails:
    print(f"\nEmail: '{e}'")
    pred_label, probs = predict_proba(e, verbose=True)
    print(f"Lớp dự đoán: {pred_label}")


Email: 'Nhận quà miễn phí hôm nay'
Xác suất spam: 0.9583
Xác suất ham: 0.0417
Lớp dự đoán: spam

Email: 'Cuộc họp diễn ra lúc 9h sáng'
Xác suất spam: 0.0333
Xác suất ham: 0.9667
Lớp dự đoán: ham

Email: 'Giảm giá 70% cho sản phẩm mới'
Xác suất spam: 0.8342
Xác suất ham: 0.1658
Lớp dự đoán: spam


Cell 8: Cho phép nhập email tùy ý để test

In [8]:
email_input = input("Nhập email để test: ")
pred_label, probs = predict_proba(email_input)
print(f"\nEmail: \n{email_input}")
print(f"Lớp dự đoán: {pred_label}")
print("Xác suất từng lớp:")
for label, p in probs.items():
    print(f"{label}: {p:.4f}")

KeyboardInterrupt: Interrupted by user